In [1]:
import pandas as pd
import numpy as np

In [2]:
root_path = './source-data/'
sub_path = 'detailed/'   # change this when it is necessary
spilits_path = 'cv/profileID/'

In [15]:
data = pd.read_csv(root_path+sub_path+'profiles.csv.xz')
data_target = pd.read_csv(root_path+sub_path+'outputs.csv.xz')

,sequenceID,position,signal
0,506_chr2,18094,0.209141
1,506_chr2,89172,0.180148
2,506_chr2,156756,0.199123
3,506_chr2,326969,0.319618
4,506_chr2,1048419,0.303342
...,...,...,...
1839134,508_chr11,134274301,-0.523370
1839135,508_chr11,134314113,-0.213680
1839136,508_chr11,134348285,-0.236960
1839137,508_chr11,134385903,-0.034680
